# Runtime test notebook

In [1]:
import rewardlm

## Data Utils

In [2]:
from rewardlm.data.data_utils import gen_benchmark_data
from rewardlm.core.GenerativeModel import GenerativeModel

generative_manager = GenerativeModel(model_id = 'EleutherAI/pythia-70m')

gen_benchmark_data(generative_manager.tokenizer)

Using MPS (Apple Silicon GPU)


Found cached dataset json (/Users/daniel/.cache/huggingface/datasets/allenai___json/allenai--real-toxicity-prompts-eb8779dd2693db47/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


TypeError: __init__() got an unexpected keyword argument 'text_col'

## Reward model

In [3]:
from rewardlm.core.RewardModel import RewardModel
reward_manager = RewardModel(model_id = 'facebook/roberta-hate-speech-dynabench-r4-target', device='mps')

### Reward tokenizer

In [6]:
reward_manager.tokenize_text('Hello, world!')

{'input_ids': tensor([[    0, 31414,     6,   232,   328,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [10]:
# Trying w/ batch text
reward_manager.tokenize_text(
    ['First sentence', 'second longer sentence'],
    padding = 'max_length', 
    max_length = 12, 
    truncation = True,
)

{'input_ids': tensor([[    0, 10993,  3645,     2,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 10815,  1181,  3645,     2,     1,     1,     1,     1,     1,
             1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

### Model itself

In [7]:
reward_manager.get_score_pair(
    prompt = reward_manager.tokenize_text('This is a prompt'),
    response = reward_manager.tokenize_text('This is the continuation of the prompt')
)

(array([0.00119799], dtype=float32), array([0.00028152], dtype=float32))

In [13]:
from torch.utils.data import DataLoader, Dataset
from rewardlm.data.CustomDatasets import ToxicityGeneratedSet
import pandas as pd

# trying w/ batch text
batch_input = reward_manager.tokenize_text(
    ['First sentence', 'second longer sentence', 'third one'],
    padding = 'max_length', 
    max_length = 12, 
    truncation = True,
)

dataset = ToxicityGeneratedSet(pd.DataFrame)
        
reward_manager.get_batch_score_pair(
    DataLoader(batch_input, batch_size=1)
)

TypeError: __init__() missing 1 required positional argument: 'tokenizer'